# Binned Rank Winrate Model

## Overview
This notebook will outline a simple method to generate predictions based on the players' most recent rank. We will bin the ranks and calculate the historic winrates between different ranks. For example, what is the historic winrate for a player ranked between 1-5th and 15th-25th? We can use this to generate predictions for every possible match up in the Australian Open. In this example we will look at both the mens and womens draw.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [5]:
# Read in both datasets
mens = pd.read_csv('data/ATP_matches_Jan_10.csv', na_values='.').dropna()
womens = pd.read_csv('data/WTA_matches_Jan_10.csv', na_values='.').dropna()

In [6]:
mens.head(2)

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_DoubleFaults,Loser_FirstServes_Won,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,02-Jan-12,Hard,First Round,106.0,97.0,0,2.0,...,3.0,21.0,33.0,13.0,26.0,1.0,3.0,15.0,49.0,49
1,Dudi Sela,Fabio Fognini,Chennai,02-Jan-12,Hard,First Round,83.0,48.0,0,2.0,...,4.0,17.0,32.0,5.0,26.0,0.0,1.0,8.0,33.0,30


Logical bins based on ranking are 1st-5th, 6th-15th, 16th-39th, 40th-100th, 101st-200th and 200th-10000th. We will use these as our groups to calculate historic winrates for.

In [7]:
best_bin = [0, 5, 15, 40, 100, 200, 10000]

# Define a function which cuts the main dfs rank columns into their bins. e.g. if a player is ranked 4th, this function
# will output their bin as "5" as this is the cutoff
def get_binned_df(df, bins):
    # Cut the rank for both the winner and the loser
    df['winner_rank_bin'] = pd.cut(df.Winner_Rank.astype(int), 
           bins=bins,
          labels=bins[1:])

    df['loser_rank_bin'] = pd.cut(df.Loser_Rank.astype(int), 
           bins=bins,
          labels=bins[1:])
    return df

# Define a function which calculates the winrates between bins and returns a dictionary of these winrates
def get_winrates_between_groups(grouped_df, bins):
    # Get win rates between groups
    win_rates = {}
    
    # Loop over the bins
    for i in bins[1:]:
        for j in bins[1:]:
            # Get the total amount of wins for each group against another group
            # e.g. 1-5th vs 15th-25th may have 49 wins and 15th-25th vs 1-5th may have 30 wins
            winner_bin_wins = grouped_df[(grouped_df.winner_rank_bin == i) & (grouped_df.loser_rank_bin == j)].Winner.values[0]
            loser_bin_losses = grouped_df[(grouped_df.loser_rank_bin == i) & (grouped_df.winner_rank_bin == j)].Winner.values[0]
            
            # From this we can calculate the win percentage
            # e.g. 1-5th would have a win percentage of 49 / (49 + 30)
            
            win_pc = winner_bin_wins / (loser_bin_losses + winner_bin_wins)
            lose_pc = loser_bin_losses / (loser_bin_losses + winner_bin_wins)
            
            # Add this percentage to the win_rates dictionary
            win_rates[str(i) + ' vs ' + str(j)] = win_pc
            win_rates[str(j) + ' vs ' + str(i)] = lose_pc
    return win_rates

In [8]:
# Create the binned ranks in a new df
binned_df = get_binned_df(mens, best_bin)

# Get count of wins and losses for each group vs other groups
mens_groups = mens.groupby(['winner_rank_bin', 'loser_rank_bin'], as_index=False).Winner.count()

In [9]:
mens_groups.head(2)

,winner_rank_bin,loser_rank_bin,Winner
0,5,5,125
1,5,15,279


In [10]:
# Get win rates dict
win_rates = get_winrates_between_groups(mens_groups, best_bin)

In [11]:
win_rates

{'100 vs 100': 0.5,
 '100 vs 10000': 0.6907356948228883,
 '100 vs 15': 0.22572002679169456,
 '100 vs 200': 0.5883458646616542,
 '100 vs 40': 0.36809470124013527,
 '100 vs 5': 0.1093525179856115,
 '10000 vs 100': 0.30926430517711173,
 '10000 vs 10000': 0.5,
 '10000 vs 15': 0.16793893129770993,
 '10000 vs 200': 0.3718518518518519,
 '10000 vs 40': 0.16715542521994134,
 '10000 vs 5': 0.03225806451612903,
 '15 vs 100': 0.7742799732083054,
 '15 vs 10000': 0.8320610687022901,
 '15 vs 15': 0.5,
 '15 vs 200': 0.8264642082429501,
 '15 vs 40': 0.6802906448683016,
 '15 vs 5': 0.23770491803278687,
 '200 vs 100': 0.4116541353383459,
 '200 vs 10000': 0.6281481481481481,
 '200 vs 15': 0.1735357917570499,
 '200 vs 200': 0.5,
 '200 vs 40': 0.29711375212224106,
 '200 vs 5': 0.09693877551020408,
 '40 vs 100': 0.6319052987598647,
 '40 vs 10000': 0.8328445747800587,
 '40 vs 15': 0.31970935513169846,
 '40 vs 200': 0.7028862478777589,
 '40 vs 40': 0.5,
 '40 vs 5': 0.18294573643410852,
 '5 vs 100': 0.890647482

In [12]:
# Read dummy submissions file
submissions = pd.read_csv('data/men_final_submission_file.csv')

# Reshape the df to long by appending the loser data to the winner data
winner_df = mens[['Winner', 'Tournament_Date', 'Winner_Rank']].rename(columns={'Winner': 'Player', 'Winner_Rank': 'Rank'})
loser_df = mens[['Loser', 'Tournament_Date', 'Loser_Rank']].rename(columns={'Loser': 'Player', 'Loser_Rank': 'Rank'})
long_df = winner_df.append(loser_df)

# Get a df of the players most recent rank based on the last game they played
recent_rankings = long_df.groupby('Player', as_index=False).last()[['Player', 'Rank']]

# Merge this data to the submissions df
submissions_with_rank = (submissions.pipe(pd.merge, 
                                          recent_rankings, 
                                          left_on='player_1', 
                                          right_on='Player', 
                                          suffixes=('', '_1'))
                         .pipe(pd.merge, recent_rankings, left_on='player_2', right_on='Player',  suffixes=('', '_2'))
                         .drop(columns=['Player', 'Player_2']))


# Get the players' rank groups
submissions_with_rank['player_1_rank'] = pd.cut(submissions_with_rank.Rank.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_2_rank'] = pd.cut(submissions_with_rank.Rank_2.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_1_vs_player_2_rank'] = submissions_with_rank.player_1_rank.astype(str) + ' vs ' + submissions_with_rank.player_2_rank.astype(str)

# Map win rates from dict to df
submissions_with_rank['player_1_win_probability'] = submissions_with_rank.player_1_vs_player_2_rank.map(win_rates).fillna(0)

In [13]:
# Create submissions df
mens_submission_df = submissions_with_rank[['player_1', 'player_2', 'player_1_win_probability']].copy()

In [14]:
mens_submission_df.head(10)

,player_1,player_2,player_1_win_probability
0,Novak Djokovic,Mitchell Krueger,0.967742
1,Taro Daniel,Mitchell Krueger,0.690736
2,Thanasi Kokkinakis,Mitchell Krueger,0.628148
3,Pablo Andujar,Mitchell Krueger,0.690736
4,Ryan Harrison,Mitchell Krueger,0.690736
5,Nicolas Jarry,Mitchell Krueger,0.690736
6,Pablo Carreno Busta,Mitchell Krueger,0.832845
7,Philipp Kohlschreiber,Mitchell Krueger,0.832845
8,Zhe Li,Mitchell Krueger,0.500000
9,Ugo Humbert,Mitchell Krueger,0.690736


Now let's repeat the process for the women's data

In [30]:
# Create the binned ranks in a new df
womens_binned_df = get_binned_df(womens, best_bin)

# Get count of wins and losses for each group vs other groups
womens_groups = womens.groupby(['winner_rank_bin', 'loser_rank_bin'], as_index=False).Winner.count()

# Get win rates dict
womens_win_rates = get_winrates_between_groups(womens_groups, best_bin)

# Read dummy submissions file
womens_submissions = pd.read_csv('data/women_final_submission_file.csv', encoding='latin1')

# Reshape the df to long by appending the loser data to the winner data
winner_df = womens[['Winner', 'Tournament_Date', 'Winner_Rank']].rename(columns={'Winner': 'Player', 'Winner_Rank': 'Rank'})
loser_df = womens[['Loser', 'Tournament_Date', 'Loser_Rank']].rename(columns={'Loser': 'Player', 'Loser_Rank': 'Rank'})
long_df = winner_df.append(loser_df)

# Get a df of the players most recent rank based on the last game they played
recent_rankings = long_df.groupby('Player', as_index=False).last()[['Player', 'Rank']]

# Merge this data to the submissions df
submissions_with_rank = (womens_submissions.pipe(pd.merge, 
                                          recent_rankings, 
                                          left_on='player_1', 
                                          right_on='Player', 
                                          suffixes=('', '_1'),
                                            how='left')
                         .pipe(pd.merge, recent_rankings, left_on='player_2', right_on='Player',  suffixes=('', '_2'), how='left')
                         .drop(columns=['Player', 'Player_2'])
                         .fillna(500)) # Fill NA with rank 500

In [32]:
# Get the players' rank groups
submissions_with_rank['player_1_rank'] = pd.cut(submissions_with_rank.Rank.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_2_rank'] = pd.cut(submissions_with_rank.Rank_2.astype(int), 
                                                bins=best_bin,
                                                labels=best_bin[1:])

submissions_with_rank['player_1_vs_player_2_rank'] = submissions_with_rank.player_1_rank.astype(str) + ' vs ' + submissions_with_rank.player_2_rank.astype(str)

# Map win rates from dict to df
submissions_with_rank['player_1_win_probability'] = submissions_with_rank.player_1_vs_player_2_rank.map(womens_win_rates).fillna(0)
womens_submission_df = submissions_with_rank[['player_1', 'player_2', 'player_1_win_probability']].copy()

In [33]:
submissions_with_rank.head(10)

,player_1,player_2,player_1_win_probability,Rank,Rank_2,player_1_rank,player_2_rank,player_1_vs_player_2_rank
0,Simona Halep,Kaia Kanepi,0.817961,1.0,44.0,5,100,5 vs 100
1,Simona Halep,Alize Cornet,0.817961,1.0,43.0,5,100,5 vs 100
2,Simona Halep,Lara Arruabarrena,0.817961,1.0,82.0,5,100,5 vs 100
3,Simona Halep,Mihaela Buzarnescu,0.690698,1.0,25.0,5,40,5 vs 40
4,Simona Halep,Carla Suarez Navarro,0.690698,1.0,23.0,5,40,5 vs 40
5,Simona Halep,Clara Burel,0.000000,1.0,500.0,5,10000,5 vs 10000
6,Simona Halep,Samantha Stosur,0.817961,1.0,73.0,5,100,5 vs 100
7,Simona Halep,Dayana Yastremska,0.817961,1.0,66.0,5,100,5 vs 100
8,Simona Halep,Eugenie Bouchard,0.817961,1.0,87.0,5,100,5 vs 100
9,Simona Halep,Shuai Peng,0.872180,1.0,125.0,5,200,5 vs 200


In [35]:
mens_submission_df.to_csv('submission/datathon_submission_mens_DataScientists_Rank_Model.csv', index=False)
womens_submission_df.to_csv('submission/datathon_submission_womens_DataScientists_Rank_Model.csv', index=False)